In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import numpy
from os import walk
from os import listdir
from os.path import isfile, join

import gensim

import random

D:\Programs\Anaconda\lib\site-packages\gensim-2.1.0-py3.5-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def get_afinn(filename):
    with open(filename) as w:
        lines = [line.split('\t') for line in w]
    afinn = {}
    for i, m in enumerate(lines):
        afinn[m[0]] = int(m[1])
    return afinn

def line_sentiment(line, afinn):
    words = line.lower().split()
    i = 0
    for each in words:
        i += afinn.get(each, 0)
    return i

def get_data(path, path2):
    filenames = []
    data = {}
    for root, dirs, files in walk(path):
        for name in dirs:
            pth = '//'.join((root, name))
            filenames.append([(pth+'//'+f, f) for f in listdir(pth) if isfile(pth+'//'+f)])
    for i in range(len(filenames[0])):
        label = filenames[0][i][1].rsplit('.',1)[0]
        for j in range(len(filenames)):
            with open(filenames[j][i][0]) as f:
                if j:
                    data[label] += [line.split('\n')[0] for line in f]
                else:
                    data[label] = [line.split('\n')[0] for line in f]
    pom = {}
    filenames = []
    for root, dirs, files in walk(path2):
        for name in dirs:
            if name == 'txt.parag':
                pth = '//'.join((root, name))
                filenames += [(pth+'//'+f, f) for f in listdir(pth) if isfile(pth+'//'+f)]

    for i in range(len(filenames)):
        with open(filenames[i][0]) as f:
            pom[filenames[i][1].split('.')[0]] = ' '.join([line.split('\n')[0] for line in f])
    for i, m in enumerate(data.get('id')):
        if i > 0:
            data['full'].append(pom.get(m))
        else:
            pom.get(m)
            data['full'] = [pom.get(m)]
    return data

def split(data, count, ratio=0.8):
    test = {}
    train = {}
    keys = data.keys()
    
    shuffle = list(range(0,count))
    random.shuffle(shuffle)
    train_size = int(count * ratio)
    test_size = count - train_size
    
    for i in shuffle[:train_size]:
        for k in keys:
            if k in train:
                train[k].append(data[k][i])
            else:
                train[k] = [data[k][i]]

    for i in shuffle[-test_size:]:
        for k in keys:
            if k in test:
                test[k].append(data[k][i])
            else:
                test[k] = [data[k][i]]
    return test, train

In [3]:
afinn = get_afinn("AFINN-111.txt")

In [4]:
data = get_data('scaledata//scaledata//', 'scaledata//scale_whole_review//')
#  id, label.3class, label.4class, rating, subj

In [5]:
test, train = split(data, len(data['id']))

In [21]:
def preprocess(data, label):
    x = []
    y = []
    wnl = WordNetLemmatizer()
    all_words = set()
    stopw = set(stopwords.words('english'))
    
    x = [[wnl.lemmatize(w) for w in word_tokenize(d.lower()) if w not in stopw] for d in data]
    y = [l for l in label]
    for d in x:
        for w in d:
            all_words.add(w)
    return all_words, x, y

In [22]:
all_wordsx, x_train, y_train = preprocess(train['full'], train['label.3class'])
all_wordsy, x_test, y_test = preprocess(test['full'], test['label.3class'])